# Sector-coupling with `pypsa`

:::{note}
If you have not yet set up Python on your computer, you can execute this tutorial in your browser via [Google Colab](https://colab.research.google.com/). Click on the rocket in the top right corner and launch "Colab". If that doesn't work download the `.ipynb` file and import it in [Google Colab](https://colab.research.google.com/).

Then install the following packages by executing the following command in a Jupyter cell at the top of the notebook.

```sh
!pip install pypsa pandas numpy matplotlib plotly highspy
```
:::

In [ ]:
import pypsa
import pandas as pd
import numpy as np
import plotly.io as pio
import plotly.offline as py
import matplotlib.pyplot as plt
pd.options.plotting.backend = "plotly"

## Previous electricity-only PyPSA model

To explore sector-coupling options with PyPSA, let's load the capacity expansion model we built for the electricity system and add sector-coupling technologies and demands on top. 

Some of the sector-coupling technologies we are going to add have multiple ouputs (e.g. CHP plants producing heat and power). PyPSA can automatically handle links have more than one input (`bus0`)
and/or output (i.e. `bus1`, `bus2`, `bus3`) with a given efficieny (`efficiency`, `efficiency2`, `efficiency3`).

In [ ]:
url = "https://tubcloud.tu-berlin.de/s/F2i54ccdfxzaDAP/download/electricity-network.nc"
n = pypsa.Network(url)

In [ ]:
n

## Hydrogen Production

The following example shows how to model the components of hydrogen storage separately, i.e. electrolysis, fuel cell and storage.

First, let's remove the simplified hydrogen storage representation:

In [ ]:
n.remove("StorageUnit", "hydrogen storage underground")

Add a separate `Bus` for the hydrogen energy carrier:

In [ ]:
n.add("Bus", "hydrogen", carrier='hydrogen')

Add a `Link` for the hydrogen electrolysis:

In [ ]:
n.add(
    "Link",
    "electrolysis",
    bus0="electricity",
    bus1="hydrogen",
    carrier="electrolysis",
    p_nom_extendable=True,
    efficiency=0.7,
    capital_cost=50e3,  # €/MW/a
)

Add a `Link` for the fuel cell which reconverts hydrogen to electricity:

In [ ]:
n.add(
    "Link",
    "fuel cell",
    bus0="hydrogen",
    bus1="electricity",
    carrier="fuel cell",
    p_nom_extendable=True,
    efficiency=0.5,
    capital_cost=120e3,  # €/MW/a
)

Add a `Store` for the hydrogen storage:

In [ ]:
n.add(
    "Store",
    "hydrogen storage",
    bus="hydrogen",
    carrier="hydrogen storage",
    capital_cost=140,  # €/MWh/a
    e_nom_extendable=True,
    e_cyclic=True,  # cyclic state of charge
)

We can also add a hydrogen demand to the hydrogen bus.

In the example below, we add a constant hydrogen demand the size of the electricity demand.

In [ ]:
p_set = n.loads_t.p_set["demand"].mean()

In [ ]:
p_set

In [ ]:
n.add("Load", "hydrogen demand", bus="hydrogen", carrier="hydrogen", p_set=p_set)  # MW

When we now optimize the model with additional hydrogen demand...

In [ ]:
n.optimize()

...we can see the individual sizing of the electrolyser, fuel cell and hydrogen storage:

In [ ]:
n.statistics.optimal_capacity().div(1e3).round(2)

Furthermore, we might want to explore the storage state of charge of the hydrogen storage and the balancing patterns:

In [ ]:
n.stores_t.e.div(1e6).plot() # TWh

## Heat Demand

For modelling simple heating systems, we create another bus and connect a load with the heat demand time series to it.

In [ ]:
n.add("Bus", "heat", carrier='heat')

In [ ]:
url = "https://tubcloud.tu-berlin.de/s/ntAA2KR6CLxD4bi/download/heat-demand.csv"
p_set = pd.read_csv(url, index_col=0, parse_dates=True).squeeze()[::3]

In [ ]:
n.add("Load", "heat demand", carrier="heat", bus="heat", p_set=p_set)

In [ ]:
n.loads_t.p_set.div(1e3).plot()

What is now missing are a few heat supply options...

## Heat pumps

To model heat pumps, first we have to calculate the **coefficient of performance (COP)** profile based on the temperature profile of the heat source.

In the example below, we calculate the COP for an air-sourced heat pump with a sink temperature of 55° C and a population-weighted ambient temperature profile for Germany.

The heat pump performance is assumed to be given by the following function:

$$
COP(\Delta T) = 6.81 - 0.121 \Delta T + 0.00063^\Delta T^2
$$
where $\Delta T = T_{sink} - T_{source}$.

In [ ]:
def cop(t_source, t_sink=55):
    delta_t = t_sink - t_source
    return 6.81 - 0.121 * delta_t + 0.000630 * delta_t**2

In [ ]:
url = "https://tubcloud.tu-berlin.de/s/7KBQ9pC5y9mdLzH/download/temperature.csv"
temp = pd.read_csv(url, index_col=0, parse_dates=True).squeeze()[::3]

In [ ]:
cop(temp).plot()

In [ ]:
plt.scatter(temp, cop(temp))
plt.xlabel("temperature [°C]")
plt.ylabel("COP [-]")

Once we have calculated the heat pump coefficient of performance, we can add the heat pump to the network as a `Link`. We use the parameter `efficiency` to incorporate the COP.

In [ ]:
n.add(
    "Link",
    "heat pump",
    carrier="heat pump",
    bus0="electricity",
    bus1="heat",
    efficiency=cop(temp),
    p_nom_extendable=True,
    capital_cost=3e5,  # €/MWe/a
)

Let's also add a resistive heater as backup technology:

In [ ]:
n.add(
    "Link",
    "resistive heater",
    carrier="resistive heater",
    bus0="electricity",
    bus1="heat",
    efficiency=0.9,
    capital_cost=1e4,  # €/MWe/a
    p_nom_extendable=True,
)

## Combined Heat-and-Power (CHP)

In the following, we are going to add gas-fired combined heat-and-power plants (CHPs). Today, these would use fossil gas, but in the example below we assume imported green methane or methanol with relatively high marginal costs. Since we have no other net emission technology, we can remove the CO$_2$ limit.

In [ ]:
n.remove("GlobalConstraint", "CO2Limit")

Then, we explicitly represent the energy carrier gas:

In [ ]:
n.add("Bus", "gas", carrier="gas")

And add a `Store` of gas, which can be depleted (up to 100 TWh) with fuel costs of 180 €/MWh.

In [ ]:
n.add(
    "Store",
    "gas storage",
    carrier="gas storage",
    e_initial=100e6,  # MWh
    e_nom=100e6,  # MWh
    bus="gas",
    marginal_cost=180,  # €/MWh_th
)

When we do this, we have to model the OCGT power plant as link which converts gas to electricity, not as generator.

In [ ]:
n.remove("Generator", "OCGT")

In [ ]:
n.add(
    "Link",
    "OCGT",
    bus0="gas",
    bus1="electricity",
    carrier="OCGT",
    p_nom_extendable=True,
    capital_cost=20000,  # €/MW/a
    efficiency=0.4,
)

Next, we are going to add a combined heat-and-power (CHP) plant with fixed heat-power ratio (i.e. backpressure operation).

:::{note}
If you want to model flexible heat-power ratios, have a look at this example: https://pypsa.readthedocs.io/en/latest/examples/power-to-gas-boiler-chp.html
:::

In [ ]:
n.add(
    "Link",
    "CHP",
    bus0="gas",
    bus1="electricity",
    bus2="heat",
    carrier="CHP",
    p_nom_extendable=True,
    capital_cost=40000,
    efficiency=0.4,
    efficiency2=0.4,
)

Now, let's optimize the current status of model:

In [ ]:
n.optimize()

The objective cost in bn€/a:

In [ ]:
n.objective / 1e9

The heat energy balance (positive is supply, negative is consumption):

In [ ]:
n.statistics.energy_balance(bus_carrier='heat').div(1e6).round(1)

The heat energy balance as a time series:

In [ ]:
n.statistics.energy_balance(aggregate_time=False, bus_carrier='heat').div(1e3).groupby("carrier").sum().T.plot()

## Long-duration heat storage

One technology of particular interest in district heating systems with large shares of renewables is long-duration thermal energy storage.

In the following, we are going to introduce a heat storage with investment cost of approximately 3 €/kWh.
The energy is not perfectly stored in water tanks. There are standing losses.
The decay of thermal energy in the heat storage is modelled through the function $1 - e^{-\frac{1}{24\tau}}$, where $\tau$ is assumed to be 180 days.
We want to see how that influences the optimal design decisions in the heating sector.

In [ ]:
n.add(
    "Store",
    "heat storage",
    bus="heat",
    carrier="heat storage",
    capital_cost=300, # roughly annuity of 3 €/kWh
    standing_loss=1 - np.exp(-1 / 24 / 180),
    e_nom_extendable=True,
)

In [ ]:
n.optimize()

The objective cost (in bn€/a) was reduced!

In [ ]:
n.objective / 1e9

The heat energy balance shows the additional losses of the heat storage and the added supply:

In [ ]:
n.statistics.energy_balance(bus_carrier='heat').div(1e6).round(1)

The heat energy balance as a time series:

In [ ]:
n.statistics.energy_balance(aggregate_time=False, bus_carrier='heat').div(1e3).groupby("carrier").sum().T.plot()

The different storage state of charge time series:

In [ ]:
n.stores_t.e.plot()

The heat energy balance shows the additional losses of the heat storage and the added supply:

In [ ]:
n.statistics.energy_balance(bus_carrier='electricity').sort_values().div(1e6).round(1)

The heat energy balance as a time series:

In [ ]:
n.statistics.energy_balance(aggregate_time=False, bus_carrier='electricity').div(1e3).groupby("carrier").sum().T.plot()

And it can tell you statistics about the capital expenditures:

In [ ]:
n.statistics.capex().groupby("carrier").sum().div(1e9).sort_values().dropna().plot.bar()

And it can tell you statistics about the operational expenditures:

In [ ]:
n.statistics.opex().groupby("carrier").sum().div(1e9).sort_values().dropna().plot.bar()

## Exercises

Explore how the model reacts to changing assumptions and available technologies. Here are a few inspirations, but choose in any order according to your interests:

- Assume underground hydrogen storage is not geographically available. Increase the cost of hydrogen storage by factor 10. How does the model react?
- Add a ground-sourced heat pump with a constant COP function of 3.5 but double the investment costs. Would this technology get built? How low would the costs need to be?
- Limit green gas imports to 10 TWh or even zero. What does the model do in periods with persistent low wind and solar feed-in but high heating demand?